In [ ]:
from pathlib import Path 

ls = [str(s) for s in Path( "/home/jonfrey/Datasets/scannet/scans/scene0000_00/color/" ).rglob("*.jpg") if int((str(s).split('/')[-1][:-4]))% 10 == 0   ]##
ls.sort(key= lambda x: int(x.split('/')[-1][:-4]))

ls = ls[int(len(ls)*0.8):]

In [ ]:
import os
import sys 
os.chdir(os.path.join(os.getenv('HOME'), 'ASL'))
sys.path.insert(0, os.getcwd())
sys.path.append(os.path.join(os.getcwd() + '/src'))
from utils_asl import LabelLoaderAuto
import numpy as np

In [ ]:
from visu import Visualizer
import imageio
visu_active = False
visu = Visualizer(os.getenv('HOME')+'/tmp', logger=None, epoch=0, store=True, num_classes=41)

In [ ]:
label_loader = LabelLoaderAuto(root_scannet= "/home/jonfrey/Datasets/scannet", confidence=0)


In [ ]:
nr_labels = 3

paths = np.zeros( (len(ls),nr_labels), dtype=object )
names = ["gt", "detectron_test_new_format", "pretrain25k"]

for i, l in enumerate( ls ):
    paths[i,0] = l.replace('color','label-filt').replace('jpg','png')
    
    base = "/home/jonfrey/Datasets/labels_generated/"
    scene = l.split('/')[-3]
    idx = l.split('/')[-1][:-4]
    
    idf = "detectron_test_new_format"
    paths[i,1] = os.path.join( base, idf, "scans", scene, idf, idx+ '.png')
    
    idf = "pretrain25k"
    paths[i,2] = os.path.join( base, idf, "scans", scene, idf, idx+ '.png')

In [ ]:
from utils_asl import png_to_label
count = 400

acc1 = [ [] for l in range( nr_labels ) ]
acc2 = [ [] for l in range( nr_labels ) ]
for i in range(paths.shape[0]):
    gt,mode = label_loader.get(paths[i,0])
    print( i,"/", paths.shape[0])
    for j in range( 0,paths.shape[1] ):
        out,mode  = label_loader.get(paths[i,j])
        
        m = gt != -1
        ac = (gt[m] == out[m]).sum() / m.sum()
        
        acc1[j].append( float( ac ) )
        
        m = (gt != 0) * ( out != 0 )
        
        ac = (gt[m] == out[m]).sum() / m.sum()
        acc2[j].append( ac )
        
        print(acc1[j][-1], names[j] )
        
        if visu_active:
            img = imageio.imread( ls[i] )
            visu.plot_detectron( img = img, label = out, tag='test', jupyter=True, store=False,alpha=0.3)


In [ ]:
label_loader = LabelLoaderAuto(root_scannet= "/home/jonfrey/Datasets/scannet", confidence=0.7)

label, _ = label_loader.get("/home/jonfrey/Datasets/labels_generated/label_pretrain25k_reprojected/scene0000_00/label_pretrain25k_reprojected/map_probs_with_confidence_0.5/0.png")
visu.plot_detectron( img = np.ones( (label.shape[0],label.shape[1],3)), label = label, tag='test', jupyter=True, store=False,alpha=0.3)

In [ ]:
import numpy as np
for j in range(nr_labels):
    print( "ACC1", names[j], np.array( acc1[j] ).mean())
    
print("")
for j in range(nr_labels):    
    print( "ACC2", names[j], np.array( acc2[j] ).mean())

In [ ]:
label_loader = LabelLoaderAuto(root_scannet= "/home/jonfrey/Datasets/scannet", confidence=0.7)

path ="/home/jonfrey/tmp/test.png"
label, _ = label_loader.get("/home/jonfrey/Datasets/labels_generated/label_pretrain25k_reprojected/scene0000_00/label_pretrain25k_reprojected/map_probs_with_confidence_0.5/0.png")
label.shape

a = np.zeros( (*label.shape,4) )
a[:,:,0] = label
a[:,:,1] = label
a = np.uint8(a)


In [ ]:
pa1 = "/home/jonfrey/Datasets/labels_generated/label_pretrain25k_reprojected/scene0000_01/label_pretrain25k_reprojected/map_probs_with_confidence_0.5/0.png"
img = imageio.imread(pa1, format='PNG-FI')

In [ ]:
%%timeit
imageio.imwrite(path, img,  format='PNG-FI', compression=9) 

In [ ]:
%%timeit
img2 = imageio.imread(path, format='PNG-FI')

In [ ]:
os.path.getsize( pa1 ) > 5000000